# CRDS For Developers


Overview
========
The rough order of the presentation will be:

1. Demonstrations of low level command line tools

a. crds certify*

b. crds refactor insert

c. crds diff         (Differences FITS, JSON, ASDF, YAML, and CRDS rules files)

d. crds submit*

e. crds uniqname*     (Uniquely names HST calibration and SYNPHOT reference files)

f. crds checksum*     (Updates FITS and CRDS rules checksums)

g. crds list          (Status / info tool useful for developers/CRDS experts)

All are embedded in the web functionality in some form.

Star (*) indicates tools the ReDCaT team uses or embeds.

2. Demonstrations of tools running as integrated web system

a. Context display and CRDS rules

b. Batch reference submission (all-in-one references submission + rules updates)

c. Submit Mappings (brief mention of doing manual rules changes)

d. Archive delivery process

e. Set Context

f. CRDS Reprocessing

g. Pipeline cache sync

h. DPAS reprocessing / repro query

Batch submission processing utilizes the command line functionality but both web-ifies the formatting and uses them in a holistic way in which certify or refactor warnings can reject or suggest rejecting a submission, differences are used to analyze updates for potential anomalies,  and files are organized and "flowed" through the tools in a unified way.

The web submission functions also uniquely add (a) cataloging by the CRDS database (b) delivery to the archive (c) implicit reprocessing triggering (c) change tracking and (b) history of past contexts.

Setup
=====
>> Setup for a particular server, JWST DEV and /grp/crds/cache for CRDS Content.

>> Another common configuration would be: CRDS_PATH=$HOME/crds_cache_dev.

In [2]:
import os
os.environ["CRDS_SERVER_URL"] = "https://jwst-crds-bit.stsci.edu"
os.environ["CRDS_PATH"] = "/grp/crds/cache"

Checking Configuration Status
------------------------------------

CRDS configuration status can be dumped out using:

In [3]:
! crds list --status

CRDS Version = '7.3.1, 7.3.1, 55463344'
CRDS_MODE = 'auto'
CRDS_PATH = '/grp/crds/cache'
CRDS_SERVER_URL = 'https://jwst-crds-bit.stsci.edu'
Cache Locking = 'enabled, multiprocessing'
Effective Context = 'jwst_0500.pmap'
Last Synced = '2019-03-18 11:19:45.350150'
Python Executable = '/Users/jmiller/anaconda3/envs/crds-env/bin/python'
Python Version = '3.6.8.final.0'
Readonly Cache = False


Obtaining test data
-----------------------

I separated the example files from the notebook source but you can obtain/recreate them like this while onsite.   This is comparatively slow so comment it out if you're re-running a lot.   With the current CRDS implementation rsync cannot help here because often you need tweaked versions of test files...  which have to be recopied anyway.

In [ ]:
!cp -v  /grp/crds/cache/references/jwst/jwst_miri_dark_0025.fits jwst_miri_dark_0025_a.fits   
!cp -v jwst_miri_dark_0025_a.fits jwst_miri_dark_0025_b.fits
!cp -v /grp/crds/cache/references/jwst/jwst_miri_dark_0057.fits .
!chmod +w *.fits
!crds_unique CRDSUNIQ *.fits

The "crds sync" tool can likewise obtain these files from the JWST OPS server,  but you should probably do that outside this notebook after setting:  

CRDS_SERVER_URL=https://jwst-crds.stsci.edu
CRDS_PATH=$HOME/crds_cache_ops

crds sync --output-dir . --files jwst_miri_dark_0025.fits jwst_miri_dark_0027.fits

CRDS Certify
===========

The CRDS certify tool/package checks CRDS reference and rules files of different formats and is embedded into the ReDCaT front end tools.

Demo of command line certify with new constraints running on obolete darks. 

Prefix >> indicates fitsverify running as a subprocess with captured stderr/stdout
fitsverify errors/warnings can also be recategorized by certify

In [3]:
!crds certify jwst_miri_dark_0025_a.fits --run-fitsverify

CRDS - INFO -  Certifying only references,  defaulting --comparison-context to operational context.
CRDS - INFO -  ########################################
CRDS - INFO -  Certifying './jwst_miri_dark_0025_a.fits' (1/1) as 'FITS' relative to context 'jwst_0500.pmap'
CRDS - INFO -  FITS file 'jwst_miri_dark_0025_a.fits' conforms to FITS standards.
INFO:CRDS: FITS file 'jwst_miri_dark_0025_a.fits' conforms to FITS standards.
CRDS - ERROR -  instrument='MIRI' type='DARK' data='./jwst_miri_dark_0025_a.fits' ::  Checking 'DQ' : Constraint (ndim(DQ_ARRAY,4)) is not satisfied.
ERROR:CRDS: instrument='MIRI' type='DARK' data='./jwst_miri_dark_0025_a.fits' ::  Checking 'DQ' : Constraint (ndim(DQ_ARRAY,4)) is not satisfied.
CRDS - ERROR -  instrument='MIRI' type='DARK' data='./jwst_miri_dark_0025_a.fits' ::  Checking 'ERR' : Constraint (ndim(ERR_ARRAY,4)) is not satisfied.
ERROR:CRDS: instrument='MIRI' type='DARK' data='./jwst_miri_dark_0025_a.fits' ::  Checking 'ERR' : Constraint (ndim(ERR_ARRAY,

Certify web output
----------------------
Related Certify web output

>> Similar output with web feedback,  colorized: https://jwst-crds-dev.stsci.edu/authenticated_result/2e126c25-6827-4832-b39e-fc27e2b0d4f9

Certify .tpn web review
---------------------------
Here is a reviewable link of the CRDS internally defined .tpn constraints: https://jwst-crds-dev.stsci.edu/browse/jwst_miri_dark_0057.fits   (open Reference Constraints .tpn panel).  These augment true Datamodels checks and CRDS automatic certify constraints based on Datamodels.

Certify --verbose
--------------------

>> Certify with verbose output to see running constraints,  useful for development / debug:

In [4]:
!crds certify jwst_miri_dark_0025_a.fits --run-fitsverify --verbose

CRDS - DEBUG -  Command: ['__main__.py', 'jwst_miri_dark_0025_a.fits', '--run-fitsverify', '--verbose']
CRDS - INFO -  Certifying only references,  defaulting --comparison-context to operational context.
CRDS - DEBUG -  CRDS JSON RPC get_server_info () -->
CRDS - DEBUG -  RPC OK 
CRDS - DEBUG -  Connected to server at 'https://jwst-crds-bit.stsci.edu'
CRDS - DEBUG -  Connected to server and computing locally, updating CRDS cache config and operational context.
CRDS - DEBUG -  CACHE Skipped update of readonly '/grp/crds/cache/config/jwst/server_config' : SERVER INFO
CRDS - DEBUG -  CACHE Skipped update of readonly '/grp/crds/cache/config/jwst/bad_files.txt' : BAD FILES LIST
CRDS - INFO -  ########################################
CRDS - INFO -  Certifying './jwst_miri_dark_0025_a.fits' (1/1) as 'FITS' relative to context 'jwst_0500.pmap'
CRDS - DEBUG -  fitsverify enabled and installled at '/Users/jmiller/anaconda3/envs/crds-env/bin/fitsverify\n'
CRDS - DEBUG -  Mapping 'META.EXPOSURE.RE

CRDS - DEBUG -  File='jwst_miri_dark_0025_a.fits' class='Character' keyword='META.INSTRUMENT.FILTER [FILTER]' value='ANY' is in ['ANY', 'CLEAR', 'F070LP', 'F070W', 'F090W', 'F1000W', 'F100LP', 'F1065C', 'F110W', 'F1130W', 'F1140C', 'F115W', 'F1280W', 'F140M', 'F140X', 'F1500W', 'F150W', 'F150W2', 'F1550C', 'F170LP', 'F1800W', 'F182M', 'F187N', 'F200W', 'F2100W', 'F210M', 'F212N', 'F2300C', 'F250M', 'F2550W', 'F2550WR', 'F277W', 'F290LP', 'F300M', 'F322W2', 'F335M', 'F356W', 'F360M', 'F380M', 'F410M', 'F430M', 'F444W', 'F460M', 'F480M', 'F560W', 'F770W', 'FLENS', 'FND', 'GR150C', 'GR150R', 'MULTIPLE', 'N/A', 'OPAQUE', 'P750L', 'WLP4']
DEBUG:CRDS: File='jwst_miri_dark_0025_a.fits' class='Character' keyword='META.INSTRUMENT.FILTER [FILTER]' value='ANY' is in ['ANY', 'CLEAR', 'F070LP', 'F070W', 'F090W', 'F1000W', 'F100LP', 'F1065C', 'F110W', 'F1130W', 'F1140C', 'F115W', 'F1280W', 'F140M', 'F140X', 'F1500W', 'F150W', 'F150W2', 'F1550C', 'F170LP', 'F1800W', 'F182M', 'F187N', 'F200W', 'F2100W

CRDS - WARNING -  NoTypeWarning : jwst.datamodels.util : model_type not found. Opening ./jwst_miri_dark_0025_a.fits as a ReferenceCubeModel
CRDS - INFO -  ########################################
INFO:CRDS: ########################################
CRDS - INFO -  7 errors
INFO:CRDS: 7 errors
CRDS - INFO -  9 warnings
INFO:CRDS: 9 warnings
CRDS - INFO -  49 infos
INFO:CRDS: 49 infos


CRDS Refactor
============
The CRDS refactor tool/module is used to update CRDS rmaps based on new reference files.

The primary refactor operation is "insert" used to add new references to rmaps.

Secondary functions are delete (removes references), set_header, and del_header for modifying rmap headers.

Adding --verbose gives more of an idea what CRDS does with checking parameters and adding new nested structure.

(adding new selector is a misnomer.  CRDS is adding a new match case which lives inside the Match() selector of this rmap.   CRDS also adds a nested UseAfter selector within that new match case...  and because this is not certify and USEAFTER was often undefined early on (before JWST even used USEAFTER,  on purpose)...  CRDS adds the USEAFTER as "UNDEFINED UNDEFINED" instead of "<DATE> <TIME>".  On the website,  certify would intercept the bad USEAFTER.   Here,  refactor intentionally passes it through so it can be fixed manually.)


In [ ]:
! cp -v /grp/crds/cache/mappings/jwst/jwst_miri_dark_0018.rmap .
! chmod +w jwst_miri_dark_0018.rmap
! crds refactor insert jwst_miri_dark_0018.rmap  new.rmap jwst_miri_dark_0025_a.fits --verbose

CRDS Diff
========

The CRDS Diff tool is used to do polymorphic differences of any CRDS files,  rules or references.

>> CRDS Diff works on reference files:

In [ ]:
!crds diff jwst_miri_dark_0025_a.fits jwst_miri_dark_0025_b.fits

>> CRDS Diff works on rules files:

In [ ]:
!crds diff ./jwst_miri_dark_0018.rmap ./new.rmap -BFQ --mapping-text-diffs

It's important to note that with standalone functions,  things like "attempting to add invalid files" are valid as shown above adding an invalid DARK file which is not a cube.

Text differences on rules provide a level of redundancy and illustrate what is really happening in the rmap.

CRDS NewContext
===============

The crds newcontext tool/module is used to generate a new .pmap and .imaps based on a set of new .rmaps.

In [ ]:
!crds newcontext --help

CRDS Submit Command Line Tool
===========================

The crds submit command line tool is a CRDS client utility which interacts with the server and is embedded in the ReDCaT tools.   

**IMPORTANT:**   Additional setup is required to run command line file submissions.   Briefly,  you need ssh access to pldmsins1,  membership in group crdsoper, a CRDS server file,  and setup for a .crds.ini files.  

See:   https://innerspace.stsci.edu/display/CRDS/Command+Line+File+Submissions

After setting up for command line installs, make the command line submission test files unique by setting a FITS header keyword:

In [4]:
! crds_unique  CRDSUNIQ ./*.fits

('./jwst_miri_dark_0025_a.fits', 'CRDSUNIQ', '=', '2019-03-18T10:53:38')
('./jwst_miri_dark_0025_b.fits', 'CRDSUNIQ', '=', '2019-03-28T10:53:38')
('./jwst_miri_dark_0057.fits', 'CRDSUNIQ', '=', '2019-04-07T10:53:38')


Next use the CRDS submit logout command to make sure we're starting from scratch,  unlocked:

In [4]:
! crds rc_submit --logout

CRDS - INFO -  =============================== setting up ===============================
CRDS - INFO -  Symbolic context 'jwst-edit' resolves to 'jwst_0503.pmap'
CRDS - INFO -  Logging out releasing lock.


# Finally,  do a command line submission

@files means there should be a file named "files" in the current directory which lists files to submit one per line or separated by spaces.

In the log output, >> indicates progress messages being emitted from the server in real time.

The same >> messages show up on a web page for through-the-web submissions.

Adding --verbose shows DEBUG log messages which show more detail, particularly web operations.

Because of the relative fragility of the command line interface and hand-overs to the web server for review and confirmation,  server actions result in STARTED, READY / BAD FILES / FAIL, and CONFIRMED / FORCED e-mails sent to the crds-servers@stsci.edu mailing list (devs) and redcat@stsci.edu.


In [ ]:
! crds rc_submit --redcat-parameters rc_submit.yaml --files jwst_miri_dark_0057.fits --keep-existing-files --monitor --wait --log-time --stats --creator "Todd Miller" --description "Small scale command line submission test." --verbosity=75

2019-03-18 11:20:01,901 - CRDS - DEBUG -  Command: ['rc_submit.py', '--redcat-parameters', 'rc_submit.yaml', '--files', 'jwst_miri_dark_0057.fits', '--keep-existing-files', '--monitor', '--wait', '--log-time', '--stats', '--creator', 'Todd Miller', '--description', 'Small scale command line submission test.', '--verbosity=75']
2019-03-18 11:20:01,901 - CRDS - INFO -  =============================== setting up ===============================
2019-03-18 11:20:01,901 - CRDS - DEBUG -  CRDS JSON RPC to https://jwst-crds-bit.stsci.edu/json/get_server_info/rc_submit-7.3.1-d2e174a0-b525-467e-ac60-a39d80cff338-00000001/ parameters () -->
2019-03-18 11:20:03,014 - CRDS - DEBUG -  RPC OK {'bad_files_list': ['jwst_fgs_distortion_0001.asdf',
                    'jwst_fgs_distortion_0002.asdf',
                    'jwst_miri_area_0001.fits',
                    'jwst_miri_cubepar_0001.fits',
                    'jwst_miri_dark_0008.rmap',
                    'jwst_miri_flat_0002.rmap',
            

                      'jwst_0231.pmap',
                      'New NIRISS SUPERBIAS,  MIRI FLATS,  NIRSPEC DFLAT + '
                      'FFLAT + SFLAT,  SYSTEM CALVER support 0.6.0'],
                     ['2016-10-03 12:42:00',
                      'jwst_0223.pmap',
                      'Since 204 --  additional files for MIRI CDP-6 DARKs, '
                      'FLATs, READNOISE, RSCD.   Also adds SYSTEM CALVER and '
                      'DATALVL s/w reference types.  Many NIRISS updates -- '
                      'AREA, DISTORTION, PHOTOM, DARK, FLAT, IPC, GAIN, '
                      'LINEARITY, READNOISE, SUPERBIAS, SATURATION.'],
                     ['2016-08-24 19:06:19',
                      'jwst_0204.pmap',
                      'Multiple FGS and NIRISS CV3 deliveries for numerous '
                      'types.  NIRSPEC SATURATION useafter date correction.  '
                      'Rmap fix for FGS,  SUBARRAY=FULL to SUBARRAY=GENERIC '
                      'for  G

2019-03-18 11:20:03,107 - CRDS - DEBUG -  RPC OK {'ingest_dir': 'pldmsins1.stsci.edu:/grp/crds/ingest/jwst/bit/jmiller',
 'submission_kinds': ['batch', 'mapping', 'reference', 'add', 'delete']}
2019-03-18 11:20:03,113 - CRDS - DEBUG -  Using reference file selection rules 'jwst-edit' defined by caller.
2019-03-18 11:20:03,113 - CRDS - INFO -  Symbolic context 'jwst-edit' resolves to 'jwst_0503.pmap'
2019-03-18 11:20:03,113 - CRDS - INFO -  Logging in aquiring lock.
2019-03-18 11:20:03,113 - CRDS - DEBUG -  -------- GET: https://jwst-crds-bit.stsci.edu/lock_status/jmiller/ --------
2019-03-18 11:20:03,113 - CRDS - DEBUG -  GET: https://jwst-crds-bit.stsci.edu/lock_status/jmiller/
2019-03-18 11:20:03,170 - CRDS - DEBUG -  ------------------------------- Response:  -------------------------------
2019-03-18 11:20:03,170 - CRDS - DEBUG -  headers:
 {'Date': 'Mon, 18 Mar 2019 15:20:03 GMT', 'Content-Length': '103', 'X-Frame-Options': 'DENY', 'Content-Type': 'application/json', 'Via': '1.1 j

2019-03-18 11:20:04,264 - CRDS - DEBUG -  ------------------------------- Response:  -------------------------------
2019-03-18 11:20:04,264 - CRDS - DEBUG -  headers:
 {'Date': 'Mon, 18 Mar 2019 15:20:04 GMT', 'Vary': 'Cookie', 'Content-Length': '30287', 'X-Frame-Options': 'DENY', 'Content-Type': 'text/html; charset=utf-8', 'Set-Cookie': 'csrftoken=FasId8uJoMMl82xqs42RSXwBJvZV57IYt31krSvzUZlLQGF69C92DndDY7XdRAMP; expires=Mon, 16-Mar-2020 15:20:04 GMT; Max-Age=31449600; Path=/', 'Via': '1.1 jwst-crds-bit.stsci.edu', 'Keep-Alive': 'timeout=5, max=93', 'Connection': 'Keep-Alive'}
2019-03-18 11:20:04,264 - CRDS - DEBUG -  ---------------------------------------------------------------------------
2019-03-18 11:20:04,264 - CRDS - DEBUG -  status_code: 200
2019-03-18 11:20:04,264 - CRDS - DEBUG -  ---------------------------------------------------------------------------
2019-03-18 11:20:04,264 - CRDS - DEBUG -  ---------------------------------------------------------------------------
20

2019-03-18 11:21:03,135 - CRDS - DEBUG -  RPC OK []
2019-03-18 11:21:06,140 - CRDS - DEBUG -  CRDS JSON RPC to https://jwst-crds-bit.stsci.edu/json/jpoll_pull_messages/rc_submit-7.3.1-d2e174a0-b525-467e-ac60-a39d80cff338-00000014/ parameters ('34d9861c-3dd4-4250-a8ea-3284ba7e1a4b', '0') -->
2019-03-18 11:21:06,194 - CRDS - DEBUG -  RPC OK []
2019-03-18 11:21:09,195 - CRDS - DEBUG -  CRDS JSON RPC to https://jwst-crds-bit.stsci.edu/json/jpoll_pull_messages/rc_submit-7.3.1-d2e174a0-b525-467e-ac60-a39d80cff338-00000015/ parameters ('34d9861c-3dd4-4250-a8ea-3284ba7e1a4b', '0') -->
2019-03-18 11:21:09,253 - CRDS - DEBUG -  RPC OK []
2019-03-18 11:21:12,258 - CRDS - DEBUG -  CRDS JSON RPC to https://jwst-crds-bit.stsci.edu/json/jpoll_pull_messages/rc_submit-7.3.1-d2e174a0-b525-467e-ac60-a39d80cff338-00000016/ parameters ('34d9861c-3dd4-4250-a8ea-3284ba7e1a4b', '0') -->
2019-03-18 11:21:12,299 - CRDS - DEBUG -  RPC OK []
2019-03-18 11:21:15,301 - CRDS - DEBUG -  CRDS JSON RPC to https://jwst-

...

2019-03-18 11:23:57,530 - CRDS - DEBUG -  RPC OK []
2019-03-18 11:24:00,531 - CRDS - DEBUG -  CRDS JSON RPC to https://jwst-crds-bit.stsci.edu/json/jpoll_pull_messages/rc_submit-7.3.1-d2e174a0-b525-467e-ac60-a39d80cff338-0000004d/ parameters ('34d9861c-3dd4-4250-a8ea-3284ba7e1a4b', '0') -->
2019-03-18 11:24:00,589 - CRDS - DEBUG -  RPC OK []
2019-03-18 11:24:03,591 - CRDS - DEBUG -  CRDS JSON RPC to https://jwst-crds-bit.stsci.edu/json/jpoll_pull_messages/rc_submit-7.3.1-d2e174a0-b525-467e-ac60-a39d80cff338-0000004e/ parameters ('34d9861c-3dd4-4250-a8ea-3284ba7e1a4b', '0') -->
2019-03-18 11:24:03,652 - CRDS - DEBUG -  RPC OK []
2019-03-18 11:24:06,656 - CRDS - DEBUG -  CRDS JSON RPC to https://jwst-crds-bit.stsci.edu/json/jpoll_pull_messages/rc_submit-7.3.1-d2e174a0-b525-467e-ac60-a39d80cff338-0000004f/ parameters ('34d9861c-3dd4-4250-a8ea-3284ba7e1a4b', '0') -->
2019-03-18 11:24:06,708 - CRDS - DEBUG -  RPC OK []
2019-03-18 11:24:09,711 - CRDS - DEBUG -  CRDS JSON RPC to https://jwst-

2019-03-18 11:25:26,114 - CRDS - DEBUG -  CRDS JSON RPC to https://jwst-crds-bit.stsci.edu/json/jpoll_pull_messages/rc_submit-7.3.1-d2e174a0-b525-467e-ac60-a39d80cff338-00000069/ parameters ('34d9861c-3dd4-4250-a8ea-3284ba7e1a4b', '0') -->
2019-03-18 11:25:26,309 - CRDS - DEBUG -  RPC OK []
2019-03-18 11:25:29,314 - CRDS - DEBUG -  CRDS JSON RPC to https://jwst-crds-bit.stsci.edu/json/jpoll_pull_messages/rc_submit-7.3.1-d2e174a0-b525-467e-ac60-a39d80cff338-0000006a/ parameters ('34d9861c-3dd4-4250-a8ea-3284ba7e1a4b', '0') -->
2019-03-18 11:25:29,373 - CRDS - DEBUG -  RPC OK []
2019-03-18 11:25:32,376 - CRDS - DEBUG -  CRDS JSON RPC to https://jwst-crds-bit.stsci.edu/json/jpoll_pull_messages/rc_submit-7.3.1-d2e174a0-b525-467e-ac60-a39d80cff338-0000006b/ parameters ('34d9861c-3dd4-4250-a8ea-3284ba7e1a4b', '0') -->
2019-03-18 11:25:32,424 - CRDS - DEBUG -  RPC OK []
2019-03-18 11:25:35,424 - CRDS - DEBUG -  CRDS JSON RPC to https://jwst-crds-bit.stsci.edu/json/jpoll_pull_messages/rc_submi

2019-03-18 11:26:51,830 - CRDS - DEBUG -  RPC OK []
2019-03-18 11:26:54,834 - CRDS - DEBUG -  CRDS JSON RPC to https://jwst-crds-bit.stsci.edu/json/jpoll_pull_messages/rc_submit-7.3.1-d2e174a0-b525-467e-ac60-a39d80cff338-00000086/ parameters ('34d9861c-3dd4-4250-a8ea-3284ba7e1a4b', '0') -->
2019-03-18 11:26:54,887 - CRDS - DEBUG -  RPC OK []
2019-03-18 11:26:57,890 - CRDS - DEBUG -  CRDS JSON RPC to https://jwst-crds-bit.stsci.edu/json/jpoll_pull_messages/rc_submit-7.3.1-d2e174a0-b525-467e-ac60-a39d80cff338-00000087/ parameters ('34d9861c-3dd4-4250-a8ea-3284ba7e1a4b', '0') -->
2019-03-18 11:26:57,943 - CRDS - DEBUG -  RPC OK []
2019-03-18 11:27:00,944 - CRDS - DEBUG -  CRDS JSON RPC to https://jwst-crds-bit.stsci.edu/json/jpoll_pull_messages/rc_submit-7.3.1-d2e174a0-b525-467e-ac60-a39d80cff338-00000088/ parameters ('34d9861c-3dd4-4250-a8ea-3284ba7e1a4b', '0') -->
2019-03-18 11:27:00,996 - CRDS - DEBUG -  RPC OK []
2019-03-18 11:27:03,996 - CRDS - DEBUG -  CRDS JSON RPC to https://jwst-

2019-03-18 11:28:22,150 - CRDS - DEBUG -  CRDS JSON RPC to https://jwst-crds-bit.stsci.edu/json/jpoll_pull_messages/rc_submit-7.3.1-d2e174a0-b525-467e-ac60-a39d80cff338-000000a2/ parameters ('34d9861c-3dd4-4250-a8ea-3284ba7e1a4b', '0') -->
2019-03-18 11:28:22,203 - CRDS - DEBUG -  RPC OK []
2019-03-18 11:28:25,205 - CRDS - DEBUG -  CRDS JSON RPC to https://jwst-crds-bit.stsci.edu/json/jpoll_pull_messages/rc_submit-7.3.1-d2e174a0-b525-467e-ac60-a39d80cff338-000000a3/ parameters ('34d9861c-3dd4-4250-a8ea-3284ba7e1a4b', '0') -->
2019-03-18 11:28:25,765 - CRDS - DEBUG -  RPC OK []
2019-03-18 11:28:28,767 - CRDS - DEBUG -  CRDS JSON RPC to https://jwst-crds-bit.stsci.edu/json/jpoll_pull_messages/rc_submit-7.3.1-d2e174a0-b525-467e-ac60-a39d80cff338-000000a4/ parameters ('34d9861c-3dd4-4250-a8ea-3284ba7e1a4b', '0') -->
2019-03-18 11:28:28,828 - CRDS - DEBUG -  RPC OK []
2019-03-18 11:28:31,829 - CRDS - DEBUG -  CRDS JSON RPC to https://jwst-crds-bit.stsci.edu/json/jpoll_pull_messages/rc_submi

2019-03-18 11:29:48,386 - CRDS - DEBUG -  RPC OK []
2019-03-18 11:29:51,389 - CRDS - DEBUG -  CRDS JSON RPC to https://jwst-crds-bit.stsci.edu/json/jpoll_pull_messages/rc_submit-7.3.1-d2e174a0-b525-467e-ac60-a39d80cff338-000000bf/ parameters ('34d9861c-3dd4-4250-a8ea-3284ba7e1a4b', '0') -->
2019-03-18 11:29:51,446 - CRDS - DEBUG -  RPC OK []
2019-03-18 11:29:54,451 - CRDS - DEBUG -  CRDS JSON RPC to https://jwst-crds-bit.stsci.edu/json/jpoll_pull_messages/rc_submit-7.3.1-d2e174a0-b525-467e-ac60-a39d80cff338-000000c0/ parameters ('34d9861c-3dd4-4250-a8ea-3284ba7e1a4b', '0') -->
2019-03-18 11:29:54,521 - CRDS - DEBUG -  RPC OK []
2019-03-18 11:29:57,523 - CRDS - DEBUG -  CRDS JSON RPC to https://jwst-crds-bit.stsci.edu/json/jpoll_pull_messages/rc_submit-7.3.1-d2e174a0-b525-467e-ac60-a39d80cff338-000000c1/ parameters ('34d9861c-3dd4-4250-a8ea-3284ba7e1a4b', '0') -->
2019-03-18 11:29:57,577 - CRDS - DEBUG -  RPC OK []
2019-03-18 11:30:00,577 - CRDS - DEBUG -  CRDS JSON RPC to https://jwst-

Because the underlying command line copy operation is "cp" or "scp" there is very little status generated during copies.   OTOH,  they are as reliable and efficient as "cp" or "scp". 

You can also use --keep-existing-files if you're getting tired of duplicate file uploads,  as long as the files are truly different.  (crds_unique intentionally adds diffs,  so if you do that,  you need to re-upload.   At present files which are bit-for-bit identical (sha1sum) are rejected so if test files are confirmed,  you can no longer fully submit them without first changing them at least a little.)

Make the test files unique again (relative to the server) for follow-on web demos:

In [ ]:
! crds_uniq CRDSUNIQ ./*.fits

Web Batch Submission
===================

Batch submissions integrate all of the command line tools to check references, generate updated rmaps, generate updated imaps and pmaps,  and deliver files into cross-mounted delivery directory for the "CRDS poller".   Batch submission also generates STARTED, READY/BAD FILES/FAIL, and CONFIRM/CANCEL e-mails.

>> Potentially we can extend this page to add addtional ReDCaT metadata:

https://jwst-crds-dev.stsci.edu/batch_submit_references/

>> Replay of Allyssa'/s submission:

https://hst-crds.stsci.edu/monitor/dc1656fb-14c9-4ace-8ef2-8552e6dee323/

>> Final results page for Allyssa's submission:

https://hst-crds.stsci.edu//display_result/f5161a22-c0cd-4229-b13e-0701a188f825

>> Show Past READY / BAD FILES pages to review as example warnings + error messages:

https://hst-crds.stsci.edu/display_result/bf0c09f6-ff04-4143-885e-4ee845de113f

https://jwst-crds-dev.stsci.edu//display_result/987613e9-b7fe-4682-8eaf-0b4bea6f3b4e

https://jwst-crds.stsci.edu/display_result/8e5c830c-fb56-4111-8b2f-b2c6348676bb

https://hst-crds-test.stsci.edu/display_result/bf3101c0-718d-4dfb-85d1-ce54b7bbf6c6

https://jwst-crds.stsci.edu//display_result/4db1dda9-c7ee-469b-94bf-c5a8f39e0e0f

https://jwst-crds.stsci.edu//display_result/4db1dda9-c7ee-469b-94bf-c5a8f39e0e0f

>> Show tracking e-mails,  particularly FAIL which has no stored result


Archiving
========

Archiving is achieved by linking new CRDS files into a delivery directory which is shared with SDP.

/ifs/crds/jwst/dev/server_files/deliveries   (temporary delivery directory,  double linked files)
/ifs/crds/jwst/dev/server_files/catalogs     (permanent copies of .cat manifest files)
/ifs/crds/jwst/dev/file_cache                (Server's internal CRDS cache, permanent server copies of new files)

A .cat manifest file lists each delivered file and is a rough indicator of progress.

Files are readonly,  but the directory is r/w for members of crdsoper.

SDP's "CRDS poller" reads manifests and archives delivered files,  renaming the .cat file to .cat_proc and/or .cat_err along the way.

When the delivery is completed,  the .cat file is deleted and CRDS considers the files archived.   There may be some lag before files are officially archived due to downstream protocol delays.

A completion e-mail is sent out by the archive.  

>> Show archive completion e-mail
>> Show ReDCaT update context request e-mail


Set Context
==========

On request,  a pipeline logs into the CRDS server and updates the default context on the Set Context page.

For simple one context transitions,  the description is taken from the submission description.

For complex multi-context transitions,  a summary needs to be constructed by a human manually.

Completing Set Context for the operational context (not edit) sets a universal default for typical users,  and signifies the context in use by the archive pipeline.

https://jwst-crds-dev.stsci.edu/set_default_context/

When the CRDS sync operation is complete,  the CRDS server automatically sends a Set Context e-mail.

>> Show CRDS Set Context e-mail

CRDS reprocessing
================

CRDS reprocessing is handled as a cron job on the CRDS server and is automatically triggered by forward moving context changes.   When the cron job notices a context transition,  crds bestrefs is run in context-to-context comparison mode on potentially affected parameters retrieved from the archive.

This is captured by the server crontab, monitor_reprocessing, and affected datasets scripts.

When the CRDS reprocessing run completes,  the CRDS server automatically sends an e-mail to DPAS containing the log output and the list of dataset IDs to reprocess.   This is sufficient to begin reprocessing but not how it is done. 

There is a race condition here where CRDS reprocessing can complete and send out IDs prior to the update of the pipeline's CRDS cache and adoption of the new context for reprocessing.  DPAS is aware of the race condition and how to handle it.  Not handling it results in reprocessing using an unchanged context.


Pipeline Cache Sync
================

The crds sync tool is used by the pipeline to maintain a local cache of all CRDS reference files.

There are written level-4 requirements not to use CRDS files before archiving.   Once the archive delivery is complete,  the CRDS server "releases" delivered files for download.   

The pipeline then runs a script (documented in the Server Workflow Guide on the server) which wraps the CRDS client's cron_sync script which in turn wraps the crds sync tool.   This was originally envisioned as a pipeline cron job,  much like the one which automatically maintains /grp/crds/cache while running on the servers.

The crds sync tool runs in time dependent on the size of the delivery or backlog of missing files (TEST servers).

If the CRDS sync tool completes without errors and successfully updates the context it "pushes" the pipeline context back up to the server for tracking under the "remote context" display visible to authenticated users.

A cron jon does a similar automatic sync of /grp/crds/cache and is likewise tracked as an important remote cache.

When the pipeline operators complete their sync,  they manually send an acknowledgement e-mail.


DPAS Use of CRDS repro query and archive reprocessing
==============================================

Once DPAS has seen the CRDS repro completion e-mail and the pipeline's CRDS cache has been updated with the new files and context definition,  DPAS runs a subclass of the CRDS client's query_affected_datasets script.   This automatically retrieves stored results from the CRDS server and dumps out the list of dataset IDs to reprocess on stdout.

In [11]:
import crds.submit.rc_submit
reload(crds.submit.rc_submit)

NameError: name 'reload' is not defined